# Train a Simplicial 2-complex convolutional neural network (SCConv)


In this notebook, we will create and train a Simplicial 2-complex convolutional neural in the simplicial complex domain, as proposed in the paper by [Bunch et. al : Simplicial 2-Complex Convolutional Neural Networks (2020)](https://openreview.net/pdf?id=Sc8glB-k6e9).


We train the model to perform

The equations of one layer of this neural network are given by:

🟥 $\quad m_{y\rightarrow x}^{(0\rightarrow 0)} = ({\tilde{A}_{\uparrow,0}})_{xy} \cdot h_y^{t,(0)} \cdot \Theta^{t,(0\rightarrow0)}$

🟥 $\quad m^{(1\rightarrow0)}_{y\rightarrow x}  = (B_1)_{xy} \cdot h_y^{t,(0)} \cdot \Theta^{t,(1\rightarrow 0)}$

🟥 $\quad m^{(0 \rightarrow 1)}_{y \rightarrow x}  = (\tilde B_1)_{xy} \cdot h_y^{t,(0)} \cdot \Theta^{t,(0 \rightarrow1)}$

🟥 $\quad m^{(1\rightarrow1)}_{y\rightarrow x} = ({\tilde{A}_{\downarrow,1}} + {\tilde{A}_{\uparrow,1}})_{xy} \cdot h_y^{t,(1)} \cdot \Theta^{t,(1\rightarrow1)}$

🟥 $\quad m^{(2\rightarrow1)}_{y \rightarrow x}  = (B_2)_{xy} \cdot h_y^{t,(2)} \cdot \Theta^{t,(2 \rightarrow1)}$

🟥 $\quad m^{(1 \rightarrow 2)}_{y \rightarrow x}  = (\tilde B_2)_{xy} \cdot h_y^{t,(1)} \cdot \Theta^{t,(1 \rightarrow 2)}$

🟥 $\quad m^{(2 \rightarrow 2)}_{y \rightarrow x}  = ({\tilde{A}_{\downarrow,2}})\_{xy} \cdot h_y^{t,(2)} \cdot \Theta^{t,(2 \rightarrow 2)}$

🟧 $\quad m_x^{(0 \rightarrow 0)}  = \sum_{y \in \mathcal{L}_\uparrow(x)} m_{y \rightarrow x}^{(0 \rightarrow 0)}$

🟧 $\quad m_x^{(1 \rightarrow 0)}  = \sum_{y \in \mathcal{C}(x)} m_{y \rightarrow x}^{(1 \rightarrow 0)}$

🟧 $\quad m_x^{(0 \rightarrow 1)}  = \sum_{y \in \mathcal{B}(x)} m_{y \rightarrow x}^{(0 \rightarrow 1)}$

🟧 $\quad m_x^{(1 \rightarrow 1)}  = \sum_{y \in (\mathcal{L}_\uparrow(x) + \mathcal{L}_\downarrow(x))} m_{y \rightarrow x}^{(1 \rightarrow 1)}$

🟧 $\quad m_x^{(2 \rightarrow 1)} = \sum_{y \in \mathcal{C}(x)} m_{y \rightarrow x}^{(2 \rightarrow 1)}$

🟧 $\quad m_x^{(1 \rightarrow 2)}  = \sum_{y \in \mathcal{B}(x)} m_{y \rightarrow x}^{(1 \rightarrow 2)}$

🟧 $\quad m_x^{(2 \rightarrow 2)}  = \sum_{y \in \mathcal{L}_\downarrow(x)} m_{y \rightarrow x}^{(2 \rightarrow 2)}$

🟩 $\quad m_x^{(0)}  = m_x^{(1\rightarrow0)}+ m_x^{(0\rightarrow0)}$

🟩 $\quad m_x^{(1)}  = m_x^{(2\rightarrow1)}+ m_x^{(1\rightarrow1)}$

🟦 $\quad h^{t+1, (0)}_x  = \sigma(m_x^{(0)})$

🟦 $\quad h^{t+1, (1)}_x  = \sigma(m_x^{(1)})$

🟦 $\quad h^{t+1, (2)}_x  = \sigma(m_x^{(2)})$


Where the notations are defined in [Papillon et al : Architectures of Topological Deep Learning: A Survey of Topological Neural Networks (2023)](https://arxiv.org/abs/2304.10031).

In [26]:
import torch
import numpy as np

from toponetx import SimplicialComplex
import toponetx.datasets.graph as graph
import toponetx.datasets as datasets

from scipy.sparse import coo_matrix
from scipy.sparse import diags


# from topomodelx.nn.simplicial.scconv_layer import SCConvLayer

# Pre-processing

## Import dataset ##

The first step is to import the dataset, shrec 16, a benchmark dataset for 3D mesh classification.

In [3]:
shrec, _ = datasets.mesh.shrec_16(size="small")

shrec = {key: np.array(value) for key, value in shrec.items()}
x_0s = shrec["node_feat"]
x_1s = shrec["edge_feat"]
x_2s = shrec["face_feat"]

ys = shrec["label"]
simplexes = shrec["complexes"]

Loading shrec 16 small dataset...

done!


In [4]:
i_complex = 0
print(
    f"The {i_complex}th simplicial complex has {x_0s[i_complex].shape[0]} nodes with features of dimension {x_0s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_1s[i_complex].shape[0]} edges with features of dimension {x_1s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_2s[i_complex].shape[0]} faces with features of dimension {x_2s[i_complex].shape[1]}."
)

The 0th simplicial complex has 252 nodes with features of dimension 6.
The 0th simplicial complex has 750 edges with features of dimension 10.
The 0th simplicial complex has 500 faces with features of dimension 7.


In [5]:
#for s in simplexes:
print(simplexes[13].up_laplacian_matrix(rank=1))

  (0, 29)	1.0
  (0, 3)	-1.0
  (0, 28)	1.0
  (0, 1)	-1.0
  (0, 0)	2.0
  (1, 34)	1.0
  (1, 2)	-1.0
  (1, 28)	-1.0
  (1, 1)	2.0
  (1, 0)	-1.0
  (2, 63)	1.0
  (2, 4)	-1.0
  (2, 34)	-1.0
  (2, 2)	2.0
  (2, 1)	-1.0
  (3, 72)	1.0
  (3, 4)	-1.0
  (3, 29)	-1.0
  (3, 3)	2.0
  (3, 0)	-1.0
  (4, 72)	-1.0
  (4, 3)	-1.0
  (4, 63)	-1.0
  (4, 4)	2.0
  (4, 2)	-1.0
  :	:
  (745, 746)	-1.0
  (745, 744)	-1.0
  (745, 745)	2.0
  (745, 732)	-1.0
  (745, 731)	1.0
  (746, 748)	1.0
  (746, 747)	-1.0
  (746, 746)	2.0
  (746, 745)	-1.0
  (746, 744)	1.0
  (747, 748)	-1.0
  (747, 746)	-1.0
  (747, 747)	2.0
  (747, 711)	-1.0
  (747, 709)	1.0
  (748, 747)	-1.0
  (748, 746)	1.0
  (748, 748)	2.0
  (748, 729)	-1.0
  (748, 727)	1.0
  (749, 729)	-1.0
  (749, 728)	1.0
  (749, 749)	2.0
  (749, 711)	-1.0
  (749, 710)	1.0


## Define neighborhood structures. ##

Now we retrieve the neighborhood structures (i.e. their representative matrices) that we will use to send messges on the domain. In this case, we need the boundary matrix (or incidence matrix) $B_1$ and the adjacency matrix $A_{\uparrow,0}$ on the nodes. For a santiy check, we show that the shape of the $B_1 = n_\text{nodes} \times n_\text{edges}$ and $A_{\uparrow,0} = n_\text{nodes} \times n_\text{nodes}$. We also convert the neighborhood structures to torch tensors.

In [27]:
def normalize_higher_order_adj(A_opt):
    """
    Args:
        A_opt is an opt that maps a j-cochain to a k-cochain.
        shape [num_of_k_simplices num_of_j_simplices]

    return:
         D^{-0.5}* (A_opt)* D^{-0.5}.
    """
    rowsum = np.array(np.abs(A_opt).sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.0
    r_mat_inv_sqrt = diags(r_inv_sqrt)
    A_opt_to = A_opt.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

    return coo_matrix(A_opt_to)


In [9]:
def get_neighborhoods(simplexes):
    B1_list = []
    B2_list = []
    up_laplacian_1_list = []
    up_laplacian_2_list = []
    down_laplacian_1_list = []
    down_laplacian_2_list = []
    for simplex in simplexes:
        B1 = simplex.incidence_matrix(rank=1, signed=False)
        B2 = simplex.incidence_matrix(rank=2, signed=False)

        up_laplacian_1 = simplex.up_laplacian_matrix(rank=0) #1
        up_laplacian_2 = simplex.up_laplacian_matrix(rank=1) #2

        down_laplacian_1 = simplex.down_laplacian_matrix(rank=1) #1
        down_laplacian_2 = simplex.down_laplacian_matrix(rank=2) #2

        incidence_1 = torch.from_numpy(B1.todense()).to_sparse()
        incidence_2 = torch.from_numpy(B2.todense()).to_sparse()

        up_laplacian_1 = torch.from_numpy(up_laplacian_1.todense()).to_sparse()
        up_laplacian_2 = torch.from_numpy(up_laplacian_2.todense()).to_sparse()

        down_laplacian_1 = torch.from_numpy(down_laplacian_1.todense()).to_sparse()
        down_laplacian_2 = torch.from_numpy(down_laplacian_2.todense()).to_sparse()

        incidence_1_list.append(incidence_1)
        incidence_2_list.append(incidence_2)
        up_laplacian_1_list.append(up_laplacian_1)
        up_laplacian_2_list.append(up_laplacian_2)
        down_laplacian_1_list.append(down_laplacian_1)
        down_laplacian_2_list.append(down_laplacian_2)

    return incidence_1_list,incidence_2_list, up_laplacian_1_list, up_laplacian_2_list, down_laplacian_1_list, down_laplacian_2_list

incidence_1_list,incidence_2_list, up_laplacian_1_list, up_laplacian_2_list, down_laplacian_1_list, down_laplacian_2_list  = get_neighborhoods(simplexes)

In [19]:
# incidence_1_list

In [40]:
adjacency_1 = simplexes[13].adjacency_matrix(rank=1, signed=False)
incidence_1 = simplexes[13].incidence_matrix(rank=1, signed=False)

# k = normalize_higher_order_adj(adjacency_1)
# print(k)

print(adjacency_1.todense().shape)
k = normalize_higher_order_adj(adjacency_1)
print(k.todense().shape)


(750, 750)
(750, 750)


## Import signal ##

Since our task will be node classification, we must retrieve an input signal on the nodes. The signal will have shape $n_\text{nodes} \times$ in_channels, where in_channels is the dimension of each cell's feature. Here, we have in_channels = channels_nodes $ = 34$. This is because the Karate dataset encodes the identity of each of the 34 nodes as a one hot encoder.

In [4]:
x_0 = []
for _, v in dataset.get_simplex_attributes("node_feat").items():
    x_0.append(v)
x_0 = torch.tensor(np.stack(x_0))
channels_nodes = x_0.shape[-1]

In [6]:
print(f"There are {x_0.shape[0]} nodes with features of dimension {x_0.shape[1]}.")

There are 34 nodes with features of dimension 2.


To load edge features, this is how we would do it (note that we will not use these features for this model, and this serves simply as a demonstration).

In [7]:
x_1 = []
for k, v in dataset.get_simplex_attributes("edge_feat").items():
    x_1.append(v)
x_1 = np.stack(x_1)

In [8]:
print(f"There are {x_1.shape[0]} edges with features of dimension {x_1.shape[1]}.")

There are 78 edges with features of dimension 2.


Similarly for face features:

In [9]:
x_2 = []
for k, v in dataset.get_simplex_attributes("face_feat").items():
    x_2.append(v)
x_2 = np.stack(x_2)

In [12]:
print(f"There are {x_2.shape[0]} faces with features of dimension {x_2.shape[1]}.")

There are 45 faces with features of dimension 2.


## Define binary labels
We retrieve the labels associated to the nodes of each input simplex. In the KarateClub dataset, two social groups emerge. So we assign binary labels to the nodes indicating of which group they are a part.

We convert the binary labels into one-hot encoder form, and keep the first four nodes' true labels for the purpose of testing.

In [13]:
y = np.array(
    [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        0,
        1,
        1,
        1,
        1,
        0,
        0,
        1,
        1,
        0,
        1,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ]
)
y_true = np.zeros((34, 2))
y_true[:, 0] = y
y_true[:, 1] = 1 - y
y_test = y_true[-4:]
y_train = y_true[:30]

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

# Create the Neural Network

Using the HSNLayer class, we create a neural network with stacked layers. A linear layer at the end produces an output with shape $n_\text{nodes} \times 2$, so we can compare with our binary labels.

In [16]:
class HSN(torch.nn.Module):
    """High Skip Network Implementation for binary node classification.

    Parameters
    ---------
    channels : int
        Dimension of features
    n_layers : int
        Amount of message passing layers.

    """

    def __init__(self, channels, n_layers=2):
        super().__init__()
        layers = []
        for _ in range(n_layers):
            layers.append(
                HSNLayer(
                    channels=channels,
                )
            )
        self.linear = torch.nn.Linear(channels, 2)
        self.layers = layers

    def forward(self, x_0, incidence_1, adjacency_0):
        """Forward computation.

        Parameters
        ---------
        x_0 : tensor
            shape = [n_nodes, channels]
            Node features.

        incidence_1 : tensor
            shape = [n_nodes, n_edges]
            Boundary matrix of rank 1.

        adjacency_0 : tensor
            shape = [n_nodes, n_nodes]
            Adjacency matrix (up) of rank 0.

        Returns
        --------
        _ : tensor
            shape = [n_nodes, 2]
            One-hot labels assigned to nodes.

        """
        for layer in self.layers:
            x_0 = layer(x_0, incidence_1, adjacency_0)
        logits = self.linear(x_0)
        return torch.softmax(logits, dim=-1)

# Train the Neural Network

We specify the model with our pre-made neighborhood structures and specify an optimizer.

In [17]:
model = HSN(
    channels=channels_nodes,
    n_layers=10,
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.4)

NameError: name 'HSNLayer' is not defined

The following cell performs the training, looping over the network for a low number of epochs.

In [13]:
test_interval = 2
num_epochs = 5
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    optimizer.zero_grad()

    y_hat = model(x_0, incidence_1, adjacency_0)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        y_hat[: len(y_train)].float(), y_train.float()
    )
    epoch_loss.append(loss.item())
    loss.backward()
    optimizer.step()

    y_pred = torch.where(y_hat > 0.5, torch.tensor(1), torch.tensor(0))
    accuracy = (y_pred[-len(y_train) :] == y_train).all(dim=1).float().mean().item()
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f} Train_acc: {accuracy:.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            y_hat_test = model(x_0, incidence_1, adjacency_0)
            y_pred_test = torch.where(
                y_hat_test > 0.5, torch.tensor(1), torch.tensor(0)
            )
            test_accuracy = (
                torch.eq(y_pred_test[-len(y_test) :], y_test)
                .all(dim=1)
                .float()
                .mean()
                .item()
            )
            print(f"Test_acc: {test_accuracy:.4f}", flush=True)

Epoch: 1 loss: 0.7181 Train_acc: 0.5667
Epoch: 2 loss: 0.7003 Train_acc: 0.5667
Test_acc: 0.0000
Epoch: 3 loss: 0.6970 Train_acc: 0.5667
Epoch: 4 loss: 0.6908 Train_acc: 0.5667
Test_acc: 0.0000
Epoch: 5 loss: 0.6825 Train_acc: 0.5667
